In [1]:
import pandas as pd                      # подключаем библиотеку pandas для работы с таблицами
import numpy as np                       # подключаем библиотеку numpy
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool, EShapCalcType, EFeaturesSelectionAlgorithm
import catboost
from matplotlib import pyplot as plt     # подключаем библиотеку pyplot для построения графиков
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-darkgrid')
palette = plt.get_cmap('Set2')

In [14]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [3]:
#pip install catboost

In [4]:
#pip install ipywidgets

In [5]:
df_build = pd.read_csv('dataset/df_build_with_app_id.csv', sep=',', index_col=0)
df_build = df_build.drop(columns=['creation_date','app_id'])

In [6]:
df_build.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 871 entries, 0 to 870
Columns: 237 entries, status to equifax_cs__cnt_open_days_from_last_delay240_inf_micr
dtypes: float64(48), int64(189)
memory usage: 1.6 MB


In [7]:
df_valid = pd.read_csv('dataset/df_valid_with_app_id.csv', sep=',', index_col=0)
df_valid = df_valid.drop(columns=['creation_date','app_id'])

In [8]:
df_valid.head()

,status,nbki_cs__cnt_open_loans_in_pandemic,nbki_cs__sum_created_loans_in_pandemic,nbki_cs__sum_open_micro_loans_in_pandemic,nbki_cs__sum_created_micro_loans_in_pandemic,nbki_cs__sum_created_receive_loans_in_pandemic,nbki_cs__sum_created_receive_micro_loans_in_pandemic,nbki_cs__cnt_created_loans_in_pandemic,nbki_cs__cnt_open_micro_loans_in_pandemic,nbki_cs__cnt_created_micro_loans_in_pandemic,...,equifax_cs__cnt_open_days_from_last_delay5_micr,equifax_cs__cnt_open_days_from_last_delay6_29_micr,equifax_cs__cnt_open_days_from_last_delay30_59_micr,equifax_cs__cnt_open_days_from_last_delay60_89_micr,equifax_cs__cnt_open_days_from_last_delay90_119_micr,equifax_cs__cnt_open_days_from_last_delay120_149_micr,equifax_cs__cnt_open_days_from_last_delay150_179_micr,equifax_cs__cnt_open_days_from_last_delay180_209_micr,equifax_cs__cnt_open_days_from_last_delay210_239_micr,equifax_cs__cnt_open_days_from_last_delay240_inf_micr
0,1.0,0,0,0,0,0,0,0,0,0,...,88,93,777,0,0,0,0,0,0,0
1,1.0,0,0,0,0,0,0,0,0,0,...,225,104,104,141,141,170,0,0,0,0
2,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0,8585,0,8585,14500,14500,2,0,2,...,38,63,84,98,243,243,243,243,317,0
4,1.0,0,0,0,0,0,0,0,0,0,...,2371,0,0,0,0,0,0,0,0,0


In [9]:
df_build.isnull().sum().sum()

0

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df_build.drop(columns='status'),\
                                                    df_build['status'],\
                                                    test_size=0.3,\
                                                    stratify=df_build['status'])

In [11]:
num_features = 15

In [13]:
# Определяем все преременные
predict_columns = X_train.columns.to_list()
# определяем категориальные переменные
cat_features_num = np.where(np.isin(X_train[X_train.columns].dtypes, ['bool', 'object']))[0]

train_pool = Pool(X_train, y_train, cat_features=cat_features_num, feature_names=list(predict_columns))
test_pool = Pool(X_test, y_test, cat_features=cat_features_num, feature_names=list(predict_columns))

model = CatBoostClassifier(iterations=round(800), eval_metric='AUC')
summary = model.select_features(
    train_pool,
    eval_set=test_pool,
    features_for_select=predict_columns,
    num_features_to_select=num_features,
    steps=10,
    algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
    shap_calc_type=EShapCalcType.Regular,
    train_final_model=False,
    logging_level='Silent',
    plot=True
)
predict_columns = summary['selected_features_names']

#w = catboost.MetricVisualizer('/AUC/')
#w.start()

display(predict_columns)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

['nbki_cs__cnt_request_microcredits_7',
 'nbki_cs__min_request_microcredits_7',
 'nbki_cs__delay30_micr',
 'nbki_cs__delay120_inf_micr',
 'nbki_cs__delay30_micr_last30days',
 'nbki_cs__delay60_90_micr_last30days',
 'nbki_cs__cnt_open_days_from_last_delay8_30_micr',
 'nbki_cs__max_delay_sum_micr',
 'equifax_cs__hist_max_created_loans_months_passed',
 'equifax_cs__cnt_request_microcredits_day',
 'equifax_cs__cnt_request_microcredits_month',
 'equifax_cs__min_request_microcredits_month',
 'equifax_cs__min_request_microcredits_year',
 'equifax_cs__max_request_microcredits_day',
 'equifax_cs__delay30_59_micr_last90days']